# Item 14: Prefer Exceptions To Returning None

In the case of dividing by zero, returning None seems natural because the result is undefined.

In [17]:
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None

You may accidentally look for any `False` equivalent value to indicate errors instead of only looking for `None`.

In [18]:
x, y = 0, 5
result = divide(x, y)
if not result:
    print('Invalid inputs')  # This is wrong!

Invalid inputs


There are two ways to reduce the chance of such errors.
The first way is to split the return value into a two-tuple. 

In [19]:
def divide(a, b):
    try:
        return True, a / b
    except ZeroDivisionError:
        return False, None


The second, better way to reduce these errors is to never return None at all. Instead, raise an exception up to the caller and make them deal with it. Here, I turn a ZeroDivisionError into a ValueError to indicate to the caller that the input values are bad:

In [20]:
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs') from e

# Item 15: Know How Closures Interact With Variable Scope

If you want to sort a list of numbers but prioritize one group of numbers to come first. A common way to do this is to pass a helper function as the key argument to a list’s sort method. 

In [21]:
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (0, x)
        return (1, x)
    values.sort(key=helper)
    
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
sort_priority(numbers, group)
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


It’d be nice if this function returned whether higher-priority items were seen at all so the user interface code can act accordingly. The following help does not work.

In [22]:
def sort_priority2(numbers, group):
    found = False
    def helper(x):
        if x in group:
            found = True  # Seems simple
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

found = sort_priority2(numbers, group)
print('Found:', found)
print(numbers)

Found: False
[2, 3, 5, 7, 1, 4, 6, 8]


The `found` variable is assigned to `True` in the `helper` closure. The closure’s assignment is treated as a new variable definition within `helper`, not as an assignment within `sort_priority2`.

The `nonlocal` statement is used to indicate that scope traversal should happen upon assignment for a specific variable name. Here is the same function again using `nonlocal`:

In [23]:
def sort_priority3(numbers, group):
    found = False
    def helper(x):
        nonlocal found
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

found = sort_priority3(numbers, group)
print('Found:', found)
print(numbers)

Found: True
[2, 3, 5, 7, 1, 4, 6, 8]


When your usage of `nonlocal` starts getting complicated, it’s better to wrap your state in a helper class. 

In [24]:
class Sorter(object):
    def __init__(self, group):
        self.group = group
        self.found = False

    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)

sorter = Sorter(group)
numbers.sort(key=sorter)
assert sorter.found is True

# Item 16: Consider Generators Instead Of Returning Lists

If you want to find the index of every word in a string.

In [25]:
def index_words_iter(text):
    if text:
        yield 0
    for index, letter in enumerate(text):
        if letter == ' ':
            yield index + 1

address = 'Four score and seven years ago...'
result = list(index_words_iter(address))
print(result[:3])

[0, 5, 11]


Here is a generator that streams input from a file one line at a time and yields outputs one word at a time. 

In [26]:
def index_file(handle):
    offset = 0
    for line in handle:
        if line:
            yield offset
        for letter in line:
            offset += 1
            if letter == ' ':
               yield offset

Here is an example of its usage:

In [30]:
from itertools import islice

with open('/tmp/address.txt', 'w') as of:
    of.write('Four score and seven years ago...\n')
    
results = []
with open('/tmp/address.txt', 'r') as f:
    it = index_file(f)
    results = islice(it, 0, 3)
    print(list(results))

[0, 5, 11]


# Item 17: Be Defensive When Iterating Over Arguments

Here is a normalization function that sums the inputs to determine the total number of each relative to the total.

In [32]:
def normalize(numbers):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

This function works when given a list.

In [33]:
visits = [15, 35, 80]
percentages = normalize(visits)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


But cannot handle a generator.

In [40]:
def read_list(l):
    for tmp in l:
        yield int(tmp)
    
visits = [15, 35, 80]
percentages = normalize(read_list(visits))
print(percentages)

[]


The cause of this behavior is that an iterator only produces its results a single time. One way around this is to accept a function that returns a new iterator each time it’s called.

In [41]:
def normalize_func(get_iter):
    total = sum(get_iter())   # New iterator
    result = []
    for value in get_iter():  # New iterator
        percent = 100 * value / total
        result.append(percent)
    return result

visits = [15, 35, 80]
percentages = normalize_func(lambda: read_list(visits))
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


The better way to achieve the same result is to provide a new container class that implements the `iterator` protocol.

In [45]:
class ReadList(object):
    def __init__(self, l):
        self.l = l

    def __iter__(self):
            for i in self.l:
                yield int(i)

This new container type works correctly when passed to the original function without any modifications.

In [48]:
visits = [15, 35, 80]
iter = ReadList(visits)
percentages = normalize(iter)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


Here is a defensive normalizer that detects if an iterator is being passed in.

In [ ]:
def normalize_defensive(numbers):
    if iter(numbers) is iter(numbers):  # An iterator -- bad!
        raise TypeError('Must supply a container')
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

# Item 18: Reduce Visual Noise With Variable Positional Arguments

A log function that takes zero or more arguments

In [50]:
def log(message, *values):  # The only difference
    if not values:
        print(message)
    else:
        values_str = ', '.join(str(x) for x in values)
        print('%s: %s' % (message, values_str))

log('My numbers are', 1, 2)
log('Hi there')  # Much better

My numbers are: 1, 2
Hi there


# Item 19: Provide Optional Behavior With Keyword Arguments

In [55]:
def flow_rate(weight_diff, time_diff, period=1):
    return (weight_diff / time_diff) * period

weight_diff = 0.5
time_diff = 3
flow_per_second = flow_rate(weight_diff, time_diff)
flow_per_hour = flow_rate(weight_diff, time_diff, period=3600)
print('flow_per_second: %r' % flow_per_second)
print('flow_per_hour: %r' % flow_per_hour)

flow_per_second: 0.16666666666666666
flow_per_hour: 600.0


# Item 20: Use None And Docstrings To Specify Dynamic Default Arguments


If you want to print logging messages that are marked with the time of the logged event. In the default case, you want the message to include the time when the function was called. 

In [68]:
import datetime
from time import sleep

def log(message, when=None):
    """Log a message with a timestamp.

    Args:
        message: Message to print.
        when: datetime of when the message occurred.
            Defaults to the present time.
    """
    when = datetime.datetime.now() if when is None else when
    print('%s: %s' % (when, message))
    
log('Hi there!')
sleep(0.1)
log('Hi again!')

2017-06-04 16:43:11.774203: Hi there!
2017-06-04 16:43:11.874702: Hi again!


# Item 21: Enforce Clarity With Keyword-Only Arguments

Here is the `safe_division` function defined to accept keyword-only arguments. The `*` symbol in the argument list indicates the end of positional arguments and the beginning of keyword-only arguments.

In [69]:
def safe_division_c(number, divisor, *,
                    ignore_overflow=False,
                    ignore_zero_division=False):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise